# Example 5-4: Calculating a Planetary Location
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, pp. 304-305

This notebook demonstrates calculating the position and velocity vectors of Jupiter with respect to the Sun.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy==0.4.1

Import `numpy` and the relevant `valladopy` modules:

In [2]:
import numpy as np
import valladopy.constants as const
import valladopy.astro.celestial.utils as utils
from valladopy.astro.twobody.frame_conversions import coe2rv
from valladopy.astro.twobody.newton import newtonm
from valladopy.mathtime.julian_date import convtime
from valladopy.mathtime.vector import rot1

## Problem Definition
---

GIVEN:&ensp;Table D-3<br>
FIND: &emsp;Position and velocity vectors of Jupiter on May 20, 1994 at 20:00 UTC, with respect to the Sun, using the mean equator and equinox of IAU-76/FK5.

In [3]:
year = 1994
month = 5
day = 20
hour = minute = second = 0
timezone = dut1 = dat = 0

## Solution
---

We begin by using the `convtime` routine to determine the Julian centuries of TDB, where:

$$
T_{TT} = \frac{JD_{TT} - 2,\!451,\!545.0}{36,\!525} \cong T_{TDB}
$$

In [4]:
*_, ttdb, _, _ = convtime(year, month, day, hour, minute, second, timezone, dut1, dat)

print(f'Julian centuries of TDB:\t{ttdb}')

Julian centuries of TDB:	-0.05619437720701812


**Algorithm 33** summarizes the process for obtaining the heliocentric position and velocity vectors.

After the Julian centuries of TDB is found, we find elements $a$, $e$, $i$, $\Omega$, $\bar{\omega}$, and $\bar{\lambda}$ from **Table D-3**.

These values find the heliocentric elements for the mean equator, mean equinox of IAU-76/FK5 and are defined as:

$$
\begin{align*}
a &= 5.202603191 + 1.913 \times 10^{-7} \ T_{TDB} \\
e &= 0.04849485 + 0.000163244 \ T_{TDB} - 4.719 \times 10^{-7} \ T_{TDB}^2 - 1.97 \times 10^{-9} \ T_{TDB}^3 \\
i &= 1.30327^{\circ} - 0.0019872 \ T_{TDB} + 3.318 \times 10^{-5} \ T_{TDB}^2 + 9.2e-8 \ T_{TDB}^3 \\
\Omega &= 100.464441^{\circ} + 0.1766828 \ T_{TDB} + 0.000903877 \ T_{TDB}^2 - 7.032 \times 10^{-6} \ T_{TDB}^3 \\
\tilde{\omega} &= 14.331309^{\circ} + 0.2155525 \ T_{TDB} + 0.00072252 \ T_{TDB}^2 - 4.59 \times 10^{-6} \ T_{TDB}^3 \\
\tilde{\lambda} &= 34.351484^{\circ} + 3034.9056746 \ T_{TDB} - 0.00008501 \ T_{TDB}^2 + 4.0 \times 10^{-9} \ T_{TDB}^3
\end{align*}
$$

In [5]:
a = 5.202603191 + 1.913e-7 * ttdb  # in AU
ecc = 0.04849485 + 0.000163244 * ttdb - 4.719e-7 * ttdb ** 2
incl = np.radians(1.303270 - 0.0019872 * ttdb + 3.318e-5 * ttdb ** 2 + 9.2e-8 * ttdb ** 3)
omega = np.radians(100.464441 + 0.1766828 * ttdb + 0.000903877 * ttdb ** 2 - 7.032e-6 * ttdb ** 3)
argpmean = np.radians(14.331309 + 0.2155525 * ttdb + 0.00072252 * ttdb ** 2 - 4.59e-6 * ttdb ** 3)
lonmean = np.radians(34.351484 + 3034.9056746 * ttdb - 0.00008501 * ttdb ** 2 + 4.0e-9 * ttdb ** 3)

print('Heliocentric elements:\n')
print(f'a = {a:.6f}\tAU')
print(f'e = {ecc:.6f}')
print(f'i = {np.degrees(incl):.6f}\tdeg')
print(f'Ω = {np.degrees(omega):.6f}\tdeg')
print(f'ω = {np.degrees(argpmean):.6f}\tdeg')
print(f'λ = {np.degrees(lonmean):.6f}\tdeg')

Heliocentric elements:

a = 5.202603	AU
e = 0.048486
i = 1.303382	deg
Ω = 100.454515	deg
ω = 14.319198	deg
λ = -136.193151	deg


We then convert these to classical elements:

$$
M = \bar{\lambda} - \bar{\omega}
$$

$$
\omega = \bar{\omega} - \bar{\Omega}
$$

In [6]:
mean = lonmean - argpmean
argp = argpmean - omega

print('Classical elements:\n')
print(f'M = {np.degrees(mean):.6f}\tdeg')
print(f'ω = {np.degrees(argp):.6f}\tdeg')

Classical elements:

M = -150.512349	deg
ω = -86.135317	deg


Then use Kepler's equation to transform the elements to yield true anomaly, and compute semiparameter:

In [7]:
_, nu = newtonm(ecc, mean)
p = a * (1 - ecc ** 2) * const.AU2KM

print(f'True anomaly = {np.degrees(nu + const.TWOPI):.6f}\tdeg')
print(f'Semiparameter = {p / const.AU2KM:.6f}\tAU')

True anomaly = 206.890953	deg
Semiparameter = 5.190373	AU


Use **Algorithm 10** (`coe2rv`) to find the position and velocity vectors in the heliocentric frame. Note, this requires using the standard gravitational parameter of the sun, and the current implementation only supports Earth's standard gravitational parameter. As a workaround, set `const.MU = const.MUSUN` and solve:

In [8]:
const.MU = const.MUSUN  # workaround - use Sun's standard gravitational parameter
r, v = coe2rv(p, ecc, incl, omega, argp, nu)

print(f'r:\t{r / const.AU2KM}\t\t\tAU')
print(f'v:\t{(v / const.AU2KM) * const.DAY2SEC}\tAU/day')

r:	[-4.08000412 -3.5738706   0.10604289]			AU
v:	[ 4.88299454e-03 -5.32576892e-03 -8.72672261e-05]	AU/day


Because the numbers are very large when we convert to km and km/s, the answer is often left in AU and AU/day. A simple rotation changes the reference to the mean equator and mean equinox of IAU-76/FK5.

The obliquity of the ecliptic ($\epsilon$) can be found using **Equation 3-69**, or can be approximated as:

$$
\epsilon = 23.439291^{\circ} - 0.013004 \ T_{TDB}
$$

In [9]:
eps = utils.obliquity_ecliptic(ttdb)

print(f'ϵ = {np.degrees(eps):.6f} deg')

ϵ = 23.440022 deg


Finally, rotate the vectors through the obliquity of the ecliptic to determine the geocentric values:

$$
\begin{align*}
\vec{r}_{XYZ(FK5)} = \text{ROT1} \ [-\epsilon] \ \vec{r}_{XYZ} \\
\vec{v}_{XYZ(FK5)} = \text{ROT1} \ [-\epsilon] \ \vec{v}_{XYZ}
\end{align*}
$$

In [10]:
reci = rot1(r, -eps)
veci = rot1(v, -eps)

print(f'r (ECI):\t{reci / const.AU2KM}\t\t\tAU')
print(f'\t\t{reci}\tkm\n')
print(f'v (ECI):\t{(veci / const.AU2KM) * const.DAY2SEC}\t\t\tAU/day')
print(f'\t\t{veci}\t\t\tkm/s')

r (ECI):	[-4.08000412 -3.32112672 -1.32435399]			AU
		[-6.10359928e+08 -4.96833485e+08 -1.98120536e+08]	km

v (ECI):	[ 0.00488299 -0.00485156 -0.0021986 ]			AU/day
		[ 8.45469429 -8.40026069 -3.80677615]			km/s


Remember this is a heliocentric vector from the Sun's center to Jupiter in the IAU-76/FK5 frame (mean equator and equinox). If you need the vector from the Earth to Jupiter, you must subtract the vector from the Sun to the Earth. You can compare the answers with those from DE-245 for the mean equator and mean equinox of IAU-76/FK-5 (see **Appendix D** for more information on obtaining these vectors).

$$
\begin{align*}
\vec{r}_{XYZ(J2000)} &= -4.068697 \ \hat{I} - 3.335668 \ \hat{J} - 1.330683 \ \hat{K} \ \ \text{AU} \\
\vec{v}_{XYZ(J2000)} &= 0.004897 \ \hat{I} - 0.004840 \ \hat{J} - 0.002194 \ \hat{K} \ \ \text{AU/day}
\end{align*}
$$